In [98]:
from music21 import *
import pretty_midi as pm
import numpy as np
rn = np.random.default_rng
us = environment.UserSettings()
us['musescoreDirectPNGPath'] = '/usr/bin/musescore'
us['musicxmlPath'] = '/usr/bin/musescore'

In [135]:
in_str = 'Abide_by_Me'
# s = corpus.parse(in_str)
s = converter.parse( "".join([in_str, '.musicxml']))
# s = converter.parse("Untitled_score-Piano.musicxml").flatten().chordify()
s.show('text')
fp = s.write('midi', "".join([in_str, '.midi']))
midi = pm.PrettyMIDI("".join([in_str, '.midi']))

{0.0} <music21.metadata.Metadata object at 0x7fc4b88295b0>
{0.0} <music21.stream.Part Violin>
    {0.0} <music21.instrument.Violin 'P1: Violin: Violin'>
    {0.0} <music21.stream.Measure 1 offset=0.0>
        {0.0} <music21.clef.TrebleClef>
        {0.0} <music21.tempo.MetronomeMark Quarter=100.0>
        {0.0} <music21.key.KeySignature of 2 flats>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.dynamics.Dynamic mp>
        {0.0} <music21.note.Note D>
        {2.0} <music21.note.Note D>
        {3.0} <music21.note.Note C>
    {0.0} <music21.dynamics.Crescendo <music21.note.Note D><music21.note.Note F>>
    {0.0} <music21.spanner.Slur <music21.note.Note D><music21.note.Note F>>
    {0.0} <music21.dynamics.Diminuendo <music21.note.Note G><music21.note.Note E->>
    {0.0} <music21.spanner.Slur <music21.note.Note G><music21.note.Note D>>
    {0.0} <music21.spanner.Slur <music21.note.Note B-><music21.note.Note B->>
    {0.0} <music21.dynamics.Crescendo <music21.note.N

In [132]:
for n in s.parts[3].flatten().recurse():
    print(n, n.offset)

P4: Contrabass: Contrabass 0.0
<music21.clef.BassClef> 0.0
<music21.key.KeySignature of 2 flats> 0.0
<music21.meter.TimeSignature 4/4> 0.0
<music21.dynamics.Dynamic mp> 0.0
<music21.note.Note B-> 0.0
<music21.dynamics.Crescendo <music21.note.Note B-><music21.note.Note D>> 0.0
<music21.spanner.Slur <music21.note.Note B-><music21.note.Note D>> 0.0
<music21.dynamics.Diminuendo <music21.note.Note E-><music21.note.Note A>> 0.0
<music21.spanner.Slur <music21.note.Note E-><music21.note.Note B->> 0.0
<music21.dynamics.Diminuendo <music21.note.Note B-><music21.note.Note E->> 0.0
<music21.dynamics.Crescendo <music21.note.Note C><music21.note.Note E->> 0.0
<music21.spanner.Ottava 8va non-transposing<music21.note.Note B-><music21.note.Note F>> 0.0
<music21.spanner.Slur <music21.note.Note C><music21.note.Note E->> 0.0
<music21.note.Note B-> 2.0
<music21.note.Note A> 3.0
<music21.note.Note G> 4.0
<music21.note.Note D> 6.0
<music21.dynamics.Dynamic f> 8.0
<music21.note.Note E-> 8.0
<music21.note.Note

In [134]:
def get_time(beat_num, tempo_curve, beat_cdf):
    # Returns a time in seconds based on a tempo curve
    # General structure:
    # 1. Find the greatest key in tempo_curve/beat_cdf which is less than beat_num, call it k
    # 2. Calculate the time as follows: elapsed_time = beat_cdf[k] + (beat_num - k) / tempo_curve[k] * 60
    # 3. Return this time
    pass

def get_dynamic_curve(score, midi):
    # General structure:
    # 1a. Find tempo markings for bpm to calculate times in seconds from beats. Store [beat_num, tempo] pairs in a 2D list called tempo_curve
    # 1b. Compute beat_cdf which is basically tempo_curve integrated (ex. stores [beat_num, total_time_elapsed_by_beat]) to avoid excessive computation
    # 2. Find all dynamic markings. Convert these to velocities as fixed points, stored as floats, using get_time() to calculate their start time in seconds
    # 3. Find all crescendos/decrescendos/szforzandos. Add start, end, and middle points as fixed points and mark them as "interpolation needed" in a separate list
    # 4. For each pair of points with interpolation needed, interpolate velocities between them linearly.
    # 5a. Store the velocity curve on the MIDI "Breath" CC (2)
    # 5b. For each note, look at the velocity curve and find the closest point - copy over the velocity

    # Parsing of lists
    tempo_markings = []
    dynamic_markings = []
    crescendos = []
    decrescendos = []
    szforzandos = []
    pass

def get_articulations(score, midi):
    # General structure:
    # 1a. Find tempo markings for bpm to calculate times in seconds from beats. Store [beat_num, tempo] pairs in a 2D list called tempo_curve
    # 1b. Compute beat_cdf which is basically tempo_curve integrated (ex. stores [beat_num, total_time_elapsed_by_beat]) to avoid excessive computation
    # 2. Check each note. If it has an articulation, set the MIDI CC of the corresponding articulation HIGH during the note time found using get_time() (max value), else, LOW (min value):
    #       CC20: Staccato
    #       CC21: Tenuto
    #       CC22: Marcato
    #       CC23: Accent
    # 3. (Assume at most one slur at a time) Check each slur. Find the time of the start and end notes and drive MIDI CC24 HIGH during the slur, ramping down linearly during the end note.
    # Parsing of lists
    tempo_markings = []
    articulations = []
    slurs = []
    pass
   

In [101]:
def randomize_note_times(midi, mean_shift, stdev_shift, mean_dur, stdev_dur):
    for inst in midi.instruments:
        # Shift note times
        for note in inst.notes:
            shift = rn().normal(loc=mean_shift, scale=(stdev_shift * stdev_shift))
            dur_var = rn().gamma(shape=mean_dur/(stdev_dur*stdev_dur), scale=stdev_dur*stdev_dur)
            dur = note.end - note.start
            start = note.start - shift
            end = start + max(0, (dur * dur_var))
            note.start = start
            note.end = end
        # Make sure note times are normalized
        inst.notes.sort(key=(lambda x : x.start))

        # Option 1: Shift all notes by position of first note
        # min_time = inst.notes[0].start
        # for note in inst.notes:
        #     note.start -= min_time
        #     note.end -= min_time

        # Option 2: Set first note only to start at 0
        inst.notes[0].start = 0

def add_screwups(midi, lambda_occur, stdev_pitch_delta):
    for inst in midi.instruments:
        # Make sure note times are normalized
        inst.notes.sort(key=(lambda x : x.start))

        occurrences = rn().poisson(lam=lambda_occur*(inst.notes[len(inst.notes)-1].end))
        for iter in range(occurrences):
            if len(inst.notes) == 0:
                break
            idx = int(rn().uniform(low=0, high=len(inst.notes)))
            screwup_len = rn().poisson(lam = 1) + 1
            end_screwup_idx = max(len(inst.notes, idx + screwup_len - 1))
            screwup_type = int(rn().uniform(low=0, high=2))
            # Didn't play notes
            if screwup_type == 0:
                del inst.notes[idx : end_screwup_idx]
            # Messed up pitches
            elif screwup_type == 1:
                pitch_delta = int(rn().normal(loc=0, scale=stdev_pitch_delta))
                for note in inst.notes[idx : end_screwup_idx]:
                    note.pitch = note.pitch + pitch_delta
                    if note.pitch < 0:
                        note.pitch = 0
                    if note.pitch > 127:
                        note.pitch = 127
                

def add_pitch_bends(midi, lambda_occur, mean_delta, stdev_delta, step_size):
    for inst in midi.instruments:
        inst.pitch_bends = []
        # Flatten note times list
        single_notes = []
        last_time = 0.0
        for note in inst.notes:
            if note.start > last_time:
                single_notes.append(note)
                last_time = note.end

        fixed_bend_points = []
        # Add fixed point pitch bends
        for note in single_notes:
            # Do 1 pitch bend at start of each note
            bend = int(rn().normal(mean_delta, stdev_delta*stdev_delta))
            if bend > 8192:
                bend = 8192
            elif bend < -8191:
                bend = -8191
            fixed_bend_points.append(pm.PitchBend(bend, note.start))
            # Add more randomly
            occurrences = rn().poisson(lam=lambda_occur*(note.end-note.start))
            for i in range(occurrences):
                time = rn().uniform(low=note.start, high=note.end)
                bend = int(rn().normal(mean_delta, stdev_delta*stdev_delta))
                fixed_bend_points.append(pm.PitchBend(bend, time))

        # Sort by time from least to greatest
        fixed_bend_points.sort(key=(lambda x : x.time))

        # Linear interpolation
        inst.pitch_bends = fixed_bend_points.copy()
        for i in range(len(fixed_bend_points) - 1):
            l_pb = fixed_bend_points[i]
            r_pb = fixed_bend_points[i+1]
            n_points_to_add = int(np.floor((r_pb.time - l_pb.time) / step_size))
            for j in range(1, n_points_to_add+1):
                time = l_pb.time + j*step_size
                bend = int(l_pb.pitch + ((r_pb.pitch - l_pb.pitch) * (j / (n_points_to_add + 1))))
                if bend > 8192:
                    bend = 8192
                elif bend < -8191:
                    bend = -8191
                inst.pitch_bends.append(pm.PitchBend(bend, time))    

def normalize_velocity(midi, vel):
    for inst in midi.instruments:
        for note in inst.notes:
            note.velocity = vel

def randomize_velocity(midi, stdev_vel):
    for inst in midi.instruments:
        for note in inst.notes:
            delta = rn().normal(0, stdev_vel * stdev_vel)
            note.velocity += delta
            if note.velocity > 127:
                note.velocity = 127
            if note.velocity < 1:
                note.velocity = 1
    

In [102]:
midi = pm.PrettyMIDI("".join([in_str, '.midi']))
randomize_note_times(midi=midi, mean_shift=0, stdev_shift=0.04, mean_dur=1, stdev_dur=0.02)
add_pitch_bends(midi=midi, lambda_occur=2, mean_delta=0, stdev_delta=500, step_size=0.01)
midi.write("".join([in_str, '_modified.midi']))